In [17]:
import os
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from torch import nn, optim
from PIL import Image
import numpy as np

# Пути к данным CelebA
# DATASET_PATH = os.getcwd()  # Укажите путь к датасету
DATASET_PATH = "../datasets/img_align_celeba"
IMAGES_PATH = os.path.join(DATASET_PATH, "img_align_celeba")
ATTRIBUTES_PATH = os.path.join(DATASET_PATH, "list_attr_celeba.csv")

# Загрузка атрибутов
attributes = pd.read_csv(ATTRIBUTES_PATH)
attributes.replace(-1, 0, inplace=True)  # Преобразуем -1 в 0 для удобства

# Оставляем только интересующие атрибуты (например, Smiling, Heavy_Makeup)
#TARGET_ATTRIBUTES = ["Smiling", "Heavy_Makeup"]
#attributes = attributes[TARGET_ATTRIBUTES]


In [18]:
attributes.set_index(attributes.columns[0], inplace=True)

In [20]:
attributes

,5_o_Clock_Shadow,Arched_Eyebrows,Attractive,Bags_Under_Eyes,Bald,Bangs,Big_Lips,Big_Nose,Black_Hair,Blond_Hair,...,Sideburns,Smiling,Straight_Hair,Wavy_Hair,Wearing_Earrings,Wearing_Hat,Wearing_Lipstick,Wearing_Necklace,Wearing_Necktie,Young
image_id,,,,,,,,,,,,,,,,,,,,,
000001.jpg,0,1,1,0,0,0,0,0,0,0,...,0,1,1,0,1,0,1,0,0,1
000002.jpg,0,0,0,1,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,1
000003.jpg,0,0,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,1
000004.jpg,0,0,1,0,0,0,0,0,0,0,...,0,0,1,0,1,0,1,1,0,1
000005.jpg,0,1,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202595.jpg,0,0,1,0,0,0,1,0,0,1,...,0,0,0,0,0,0,1,0,0,1
202596.jpg,0,0,0,0,0,1,1,0,0,1,...,0,1,1,0,0,0,0,0,0,1
202597.jpg,0,0,0,0,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,1


In [4]:

# Настройки
IMAGE_SIZE = 128
BATCH_SIZE = 64
EPOCHS = 5
LEARNING_RATE = 0.001
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Трансформации изображений
transform = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
])

# Класс для работы с датасетом
class CelebADataset(Dataset):
    def __init__(self, image_dir, attributes, transform=None):
        self.image_dir = image_dir
        self.attributes = attributes
        self.transform = transform
        self.image_names = attributes.index.tolist()
        self.labels = attributes.values

    def __len__(self):
        return len(self.image_names)

    def __getitem__(self, idx):
        img_path = os.path.join(self.image_dir, self.image_names[idx])
        image = Image.open(img_path).convert("RGB")
        label = torch.tensor(self.labels[idx], dtype=torch.float32)
        if self.transform:
            image = self.transform(image)
        return image, label



In [5]:
# Создание датасета и DataLoader'ов
dataset = CelebADataset(IMAGES_PATH, attributes, transform=transform)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# Определение модели
class PersonalityNet(nn.Module):
    def __init__(self, num_classes):
        super(PersonalityNet, self).__init__()
        self.model = models.resnet18(pretrained=True)
        self.model.fc = nn.Linear(self.model.fc.in_features, num_classes)

    def forward(self, x):
        return torch.sigmoid(self.model(x))

model = PersonalityNet(40).to(DEVICE)

# Определение функции потерь и оптимизатора
criterion = nn.BCELoss()  # Поскольку задача многоклассовой классификации
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)



/home/balezz/PycharmProjects/VKR_2026/.venv/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/balezz/PycharmProjects/VKR_2026/.venv/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [6]:
# Функции обучения и тестирования
def train_model(model, train_loader, criterion, optimizer, device):
    model.train()
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

def evaluate_model(model, test_loader, device):
    model.eval()
    all_labels = []
    all_predictions = []
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            all_predictions.append(outputs.cpu())
            all_labels.append(labels.cpu())
    return torch.cat(all_predictions), torch.cat(all_labels)

In [8]:
# Обучение модели
for epoch in range(EPOCHS):
    train_model(model, train_loader, criterion, optimizer, DEVICE)
    print(f"Epoch {epoch + 1}/{EPOCHS} completed")

# Оценка модели
predictions, labels = evaluate_model(model, test_loader, DEVICE)
print("Модель обучена. Примерные предсказания готовы.")

Epoch 1/5 completed
Epoch 2/5 completed
Epoch 3/5 completed
Epoch 4/5 completed
Epoch 5/5 completed
Модель обучена. Примерные предсказания готовы.


In [15]:
MODEL_SAVE_PATH = "personality_net_v0.pth"
torch.save(model.state_dict(), MODEL_SAVE_PATH)


## Test model

In [22]:
from PIL import Image
import torch
from torchvision import transforms

# Путь к сохранённой модели и изображению

TEST_IMAGE_PATH = os.path.join(DATASET_PATH, "test.jpg")

# Загрузка сохранённой модели
loaded_model = PersonalityNet(40).to(DEVICE)
loaded_model.load_state_dict(torch.load(MODEL_SAVE_PATH, map_location=DEVICE))
loaded_model.eval()

# Трансформации, используемые для тестового изображения
test_transform = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
])

# Загрузка и предобработка изображения
image = Image.open(TEST_IMAGE_PATH).convert("RGB")
input_tensor = test_transform(image).unsqueeze(0).to(DEVICE)  # Добавляем batch-измерение

# Прогнозирование
with torch.no_grad():
    prediction = loaded_model(input_tensor)
    prediction = prediction.cpu().numpy().flatten()  

print("Предсказание модели для изображения:")
print(prediction)


Предсказание модели для изображения:
[1.13715520e-02 9.51491122e-04 4.38166456e-03 9.62925032e-02
 8.39132408e-04 5.38342632e-03 5.07408753e-02 1.73532426e-01
 2.04814628e-01 3.09379720e-05 2.11657323e-02 2.12485865e-02
 3.22130974e-03 2.47512595e-03 1.77745984e-04 3.88416499e-02
 1.09640900e-02 6.18836726e-04 8.46694689e-04 1.30326976e-03
 9.65371251e-01 5.23208175e-03 9.40306112e-02 4.00123969e-02
 5.33222497e-01 1.35384244e-03 7.48162332e-04 7.52566829e-02
 5.79895265e-02 1.04896817e-05 5.69191109e-03 1.03361315e-04
 3.39427441e-02 1.78283662e-01 1.93490032e-02 9.56028886e-03
 1.24216289e-03 6.81092078e-03 4.24492359e-03 2.78982729e-01]


/tmp/ipykernel_72858/2162012460.py:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  loaded_model.load_state_dict(torch.load(MODEL_SAVE_PATH, map_location=DEVICE))


In [23]:
attributes.columns.tolist()

['5_o_Clock_Shadow',
 'Arched_Eyebrows',
 'Attractive',
 'Bags_Under_Eyes',
 'Bald',
 'Bangs',
 'Big_Lips',
 'Big_Nose',
 'Black_Hair',
 'Blond_Hair',
 'Blurry',
 'Brown_Hair',
 'Bushy_Eyebrows',
 'Chubby',
 'Double_Chin',
 'Eyeglasses',
 'Goatee',
 'Gray_Hair',
 'Heavy_Makeup',
 'High_Cheekbones',
 'Male',
 'Mouth_Slightly_Open',
 'Mustache',
 'Narrow_Eyes',
 'No_Beard',
 'Oval_Face',
 'Pale_Skin',
 'Pointy_Nose',
 'Receding_Hairline',
 'Rosy_Cheeks',
 'Sideburns',
 'Smiling',
 'Straight_Hair',
 'Wavy_Hair',
 'Wearing_Earrings',
 'Wearing_Hat',
 'Wearing_Lipstick',
 'Wearing_Necklace',
 'Wearing_Necktie',
 'Young']

In [24]:
attribute_names = attributes.columns.tolist()  

decoded_predictions = {attribute: prob for attribute, prob in zip(attribute_names, prediction)}

positive_attributes = {attr: prob for attr, prob in decoded_predictions.items() if prob > 0.3}

print("Полные предсказания:")
print(decoded_predictions)

print("\nВероятные положительные атрибуты (prob > 0.5):")
print(positive_attributes)


Полные предсказания:
{'5_o_Clock_Shadow': np.float32(0.011371552), 'Arched_Eyebrows': np.float32(0.0009514911), 'Attractive': np.float32(0.0043816646), 'Bags_Under_Eyes': np.float32(0.0962925), 'Bald': np.float32(0.0008391324), 'Bangs': np.float32(0.0053834263), 'Big_Lips': np.float32(0.050740875), 'Big_Nose': np.float32(0.17353243), 'Black_Hair': np.float32(0.20481463), 'Blond_Hair': np.float32(3.0937972e-05), 'Blurry': np.float32(0.021165732), 'Brown_Hair': np.float32(0.021248586), 'Bushy_Eyebrows': np.float32(0.0032213097), 'Chubby': np.float32(0.002475126), 'Double_Chin': np.float32(0.00017774598), 'Eyeglasses': np.float32(0.03884165), 'Goatee': np.float32(0.01096409), 'Gray_Hair': np.float32(0.0006188367), 'Heavy_Makeup': np.float32(0.0008466947), 'High_Cheekbones': np.float32(0.0013032698), 'Male': np.float32(0.96537125), 'Mouth_Slightly_Open': np.float32(0.0052320817), 'Mustache': np.float32(0.09403061), 'Narrow_Eyes': np.float32(0.040012397), 'No_Beard': np.float32(0.5332225), 